In [17]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML

# load and display data
data = pd.read_csv('Case1_Data.csv')
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# display(data)

In [18]:
# split data in training / prediction based of Y value
training = data.loc[data['Y'].notnull()]
prediction = data.loc[data['Y'].isnull()]

print ('We have', len(training), 'training observations and', len(prediction), 'prediction observations')

('We have', 100, 'training observations and', 1000, 'prediction observations')


In [44]:
train_y = training['Y'].as_matrix()
train_X = training[[col for col in training.columns if col != 'Y']].as_matrix()


TypeError: as_matrix() got an unexpected keyword argument 'dtype'

## Printing a row to verify that the function removes the nan

In [20]:
print (train_X[:,16])

[3.97285392222803 -1.13609266459014 2.24472023894437 3.2882903537022297
 0.16458526517902403 3.0677475438827 2.73687621565302 1.9468708682615603
 4.954474902144209 nan -0.335626460992474 1.85575908022567 3.63276609676462
 1.65256467549402 0.267932078229213 1.9439147635345602 2.88629888715096
 4.61877588054924 0.8539508879783809 2.9116739565293903 4.47021894031954
 1.17889064057854 2.5869278824593 2.46381501851865 3.18541109901115
 1.4022002381594598 1.4153961256394199 2.75423679935718 2.0671853432220804
 3.1212350531801403 2.75365312974546 6.54158681741213 4.1578057398381505
 2.23284956506475 2.57066955088036 -0.27893370759489694 3.11663723168109
 4.118628175442661 3.7214264254542 0.559071901727219 1.2533074956961598
 5.389324936875499 0.7400425005617741 1.6480763906526001 2.03568391354468
 2.8378198587654797 1.59082109735179 4.48458871221162 7.068600776489661
 2.29705355614881 1.17765744653979 -0.57653056682252 2.48757006009932
 3.566022771139 4.052860618456431 1.39793797903212 1.4013

In [21]:
def missing_predictor_as_mean(training_data):
    data = np.copy(training_data)
    for i, column in enumerate(data.T):
            # column = train_X[column].as_matrix()
            column_without_nan = column[~np.isnan(np.array(column, dtype=float))]
            column_mean = column_without_nan.mean()
            # print( column_without_nan.mean() )
            for predictor_index in range(len(column)):
                if pd.isnull(np.array(column[predictor_index], dtype=float)):
                    column[predictor_index] = column_mean

            data.T[i, :] = column
    return data


In [41]:
def missing_predictor_as_value(data_set, value):
    data = np.copy(data_set, dtype=float)
    data[pd.isnull(np.array(data_set, dtype=float))] = value
    print 'data', data.dtype
    return data

#### We should consider to nomralize to zero mean and unit variance. For KNN <br>
from sklearn import preprocessing as preproc <br>
X = preproc.scale(X) # Normalize to zero mean and unit variance

In [42]:
from scipy.spatial import distance
def missing_predictor_as_knn(K, training_data, clean_data):
    data = np.copy(training_data)
    clean_data = clean_data
    n, p = training_data.shape
    # row major
    for i, row in enumerate(data):
        distances = np.zeros(n)
        for j in range(p):
            if pd.isnull(np.array(row[j], dtype=float)):
                # get the index of the nan
                # if the value is nan then we want to calculate the knn
                for k in range(n):
                    distances[k] = distance.euclidean(clean_data[i, :], clean_data[k, :])
                    
                # Calculate the estimated value of the NaN
                index = np.argsort(distances)[1:(K+1)]
                wt = sum(distances[index])
                W = distances[index] / wt
                
                nearest_neighbours = clean_data[index, j]
                print 'nn', nearest_neighbours.dtype
                print 'W', W.dtype

                value = np.matmul(W, nearest_neighbours)
                data[i,j] = value

    return data

## Checking if the Nan is removed:
### Total of 43 Nans

In [43]:
data_mean = missing_predictor_as_mean(train_X[:,:-1])
clean_data = missing_predictor_as_value(train_X[:,:-1], 0)
data_knn = missing_predictor_as_knn(2, train_X[:,:-1], clean_data)
print (train_X[:,16])
print (data_knn[:,16])

TypeError: copy() got an unexpected keyword argument 'dtype'